In [64]:
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import os

def downloadImage(url, save_location):
    # Send a HTTP request to the URL of the image
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create a BytesIO object
        img_data = BytesIO(response.content)

        # Open the image
        img = Image.open(img_data)

        # Ensure that the directory exists; if not, create it
        os.makedirs(os.path.dirname(save_location), exist_ok=True)

        # Save the image
        img.save(save_location)
    else:
        print(f'Error: Received status code {response.status_code} when trying to get {url}')


In [65]:
import io
import os
import openai
from PIL import Image

openai.api_key ="
def generateRefrenceImage(className, imagePath, output_path, file_name, numImages=1):
    # Open the JPEG image
    with Image.open(imagePath) as im:
        im = im.resize((415, 415))
        # Convert the image to RGBA format (with alpha channel)
        rgba_im = im.convert("RGBA")

        # Create a BytesIO object and save the image data to it
        img_byte_arr = io.BytesIO()
        rgba_im.save(img_byte_arr, format='PNG')

        # Move to the beginning of the BytesIO object
        img_byte_arr.seek(0)

        response = openai.Image.create_variation(
            image=img_byte_arr,
            n=numImages,
            size="256x256",
        )
        
        file_name = file_name[:-4]
        for i, generatedImg in enumerate(response.data):
            fileName = "augmented_" + file_name +"_" +str(i) +".jpg"
            save_dir = output_path +"/" + fileName;
            downloadImage(generatedImg.url, save_dir)

In [67]:
import os
from PIL import Image

base_folder_path = "data/training_set"
output_folder_path = "data/augmented_training_set"

classes = ["cats", "dogs"]

for class_name in classes:
    print("doing class:", class_name)
    folder_path = os.path.join(base_folder_path, class_name)
    output_path = os.path.join(output_folder_path,class_name)
    image_files = os.listdir(folder_path)

    for file_name in image_files:
        image_path = os.path.join(folder_path, file_name)
        try:
            with Image.open(image_path) as image:
                generateRefrenceImage(class_name, image_path, output_path, file_name, 3)
        except UnidentifiedImageError as e:
            print(f"Error: {e}. Skipping file {file_name}.")
            continue
            

print("completed generating images!")

doing class: cats
